#Used Libiraies


In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import cv2
import os
import imghdr
import random
import pickle
import re
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.models import load_model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense,GlobalMaxPool2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers, models, regularizers, optimizers, losses

#Data preparation

Import The dataset from Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
test_path='/content/drive/MyDrive/test'
train_path='/content/drive/MyDrive/train'

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Apply PreProcessing on Training data


In [ ]:
def Train_data_preprocessing_(path):
    """
    ImageDataGenerator is a class in keras.preprocessing that generates batches of images data
    with real-time data augmentation.

    Note : "real-time" indicates that these transformations are applied dynamically and on the fly,
    just before each training batch is fed into the neural network to enhance the model's ability to generalize to new, unseen data.

    This function take a path argument and returns two datasets: train_dataset and Validation_dataset
    The function applies the train test split on the data after the preprocessing
    class_mode: the type of classification problem.
    """
    image_generator = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2, )

    # Train & Test Split
    train_dataset = image_generator.flow_from_directory(batch_size=32,directory=path,shuffle=True,target_size=(150, 150),
                                                        subset="training",class_mode='categorical')

    validation_dataset = image_generator.flow_from_directory(batch_size=32, directory=path,shuffle=True, target_size=(150, 150),
                                                             subset="validation",class_mode='categorical')
    return train_dataset, validation_dataset

train , validation = Train_data_preprocessing_(train_path)

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.


Apply Preprocessing on Testing data


In [ ]:
def preprocessed_test_data(path):
    """
    This function processes test images in (test_path). It reads each image,
    converts the color space from BGR to RGB, resizes the image to (100, 100),and normalizes the pixel values.
    The processed test images and their names are then stored in the data and image_name lists,
    which are returned at the end of the function.

    Note  : we stored the image name because we will use it in the final csv file
    """
    data = []
    image_name = []
    print("loading test images")
    """
     This loop Iterates through the test classes from the given path and read each image using openCV
    """
    for img in os.listdir(path):
        image_name.append(img)
        test_img_array = cv2.imread(os.path.join(path, img))
        if test_img_array is not None:
          # BGR to RGB
          test_img_array = cv2.cvtColor(test_img_array, cv2.COLOR_BGR2RGB)
          # Resize
          test_img_array = cv2.resize(test_img_array, (150, 150))
          # Normalization
          test_img_array = test_img_array.astype(np.float32)
          test_img_array=test_img_array/255

          # test_img_array = (test_img_array - np.min(test_img_array)) / (np.max(test_img_array) - np.min(test_img_array))*255
          ###############################
          data.append(test_img_array)
        else :
          print(f"Failed to read image: {os.path.join(path, img)}")

    print("Done.")
    print("Total test images:", len(data))
    print("------------------------------")
    return data, image_name

test , images_names= preprocessed_test_data(test_path)


loading test images
Done.
Total test images: 100
------------------------------


#CNN Models

First Model

In [ ]:
def cnn_model_1(train_data, val_data, epochs):
    model = Sequential()

    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    #model.add(Dropout(0.3))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    #model.add(Dropout(0.4))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    #model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    #model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3)))
    model.add(GlobalMaxPool2D())
    #model.add(BatchNormalization())
    #model.add(Dropout(0.4))



    #model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))



    model.add(Dense(5, activation='softmax'))



    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    # Define model checkpoint callback
    model_checkpoint = ModelCheckpoint('model_1.h5', monitor='val_loss', save_best_only=True)

    print("Start training...")
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[early_stopping, model_checkpoint])
    print("Finished training...")

    return model

cnn_model_1_1 = cnn_model_1(train, validation, 100)

Seconed Model

In [ ]:
def cnn_model_2(train_data, val_data, epochs):
    model = Sequential()

    model.add(Conv2D(12, (5, 5), activation='relu', kernel_initializer='he_uniform', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(2))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(500, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(400, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(300, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(200, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(50, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(10, activation='relu', kernel_initializer='he_uniform'))

    model.add(Dense(5, activation='softmax'))

    # Use a learning rate scheduler
    def lr_schedule(epoch):
        lr = 0.001
        if epoch > 50:
            lr *= 0.5
        if epoch > 100:
            lr *= 0.5
        return lr

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Define model checkpoint callback
    model_checkpoint = ModelCheckpoint('model_2.h5', monitor='val_loss', save_best_only=True)

    # Define learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(lr_schedule)

    print("Start training...")
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[early_stopping, model_checkpoint, lr_scheduler])
    print("Finished training...")

    return model

cnn_model_2_2 = cnn_model_2(train, validation, 300)

AlexNet Model

In [ ]:
def build_alexnet(input_shape=(150, 150, 3), num_classes=5):
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Convolutional Block 2
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Convolutional Block 3
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def train_alexnet(model, train_data, test_data, epochs=100):
    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Define model checkpoint callback to save the best weights
    model_checkpoint = ModelCheckpoint('alexnet_best22.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

    # Training
    print("Start training...")
    model.fit(train_data, epochs=epochs, validation_data=test_data, callbacks=[early_stopping, model_checkpoint], verbose=1)
    print("Finished training...")

    return model

input_shape = (150, 150, 3)
num_classes = 5

# Build and train the model
alexnet_model = build_alexnet(input_shape, num_classes)
trained_model = train_alexnet(alexnet_model, train, test, epochs=100)

Testing the model

In [ ]:
def test_model(model, test_data):
    c = 1
    labels = []
    print("Predictions:")
    for img in test_data:
        print("image:", c)
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)
        # Convert the prediction array to a list for formatting
        prediction_list = prediction.squeeze().tolist()

        print(f'1: {prediction_list[0] * 100:.2f}%, 2: {prediction_list[1] * 100:.2f}%, '
              f'3: {prediction_list[2] * 100:.2f}%, 4: {prediction_list[3] * 100:.2f}%, '
              f'5: {prediction_list[4] * 100:.2f}%')

        labels.append(prediction.argmax())

        c += 1
    return labels

model = load_model('exampel.h5')
test_data_labels  = test_model(model , test)

Predictions:
image: 1
1/1 [==============================] - 0s 86ms/step
1: 11.46%, 2: 44.53%, 3: 12.81%, 4: 30.57%, 5: 0.63%
image: 2
1/1 [==============================] - 0s 22ms/step
1: 10.68%, 2: 42.36%, 3: 20.63%, 4: 25.73%, 5: 0.60%
image: 3
1/1 [==============================] - 0s 19ms/step
1: 7.72%, 2: 7.65%, 3: 77.64%, 4: 1.19%, 5: 5.80%
image: 4
1/1 [==============================] - 0s 21ms/step
1: 10.88%, 2: 17.76%, 3: 62.84%, 4: 4.96%, 5: 3.56%
image: 5
1/1 [==============================] - 0s 20ms/step
1: 3.37%, 2: 5.95%, 3: 87.38%, 4: 0.41%, 5: 2.89%
image: 6
1/1 [==============================] - 0s 18ms/step
1: 30.66%, 2: 6.25%, 3: 14.85%, 4: 4.23%, 5: 44.00%
image: 7
1/1 [==============================] - 0s 19ms/step
1: 6.22%, 2: 30.47%, 3: 0.51%, 4: 62.79%, 5: 0.02%
image: 8
1/1 [==============================] - 0s 20ms/step
1: 4.44%, 2: 7.71%, 3: 84.63%, 4: 0.97%, 5: 2.25%
image: 9
1/1 [==============================] - 0s 22ms/step
1: 7.11%, 2: 50.36%, 3: 4.5

Converting to csv

In [ ]:
def generate_csv(classified_images):
    fields = ['image_id', 'label']
    with open('try', 'w') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(classified_images)

Classified_Images = list(zip(images_names, test_data_labels))
generate_csv(Classified_Images)

Apply required format

In [ ]:
def formating (path):
  # Read the CSV file into a DataFrame
  file_csv = pd.read_csv(path)

  # Assuming the second column is named 'column_name', replace the values
  file_csv['label'] = file_csv['label'] + 1
  file_csv['image_id'] = file_csv['image_id'].str.replace('.jpeg', '')
  file_csv['image_id'] = file_csv['image_id'].str.replace('.png', '')



  # Save the modified DataFrame back to the CSV file
  file_csv.to_csv(path, index=False)
  return file_csv
# call the function and give it te path
formating('/content/try')


<ipython-input-55-7b897b952d2e>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  file_csv['image_id'] = file_csv['image_id'].str.replace('.jpeg', '')
<ipython-input-55-7b897b952d2e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  file_csv['image_id'] = file_csv['image_id'].str.replace('.png', '')


,image_id,label
0,2044,2
1,1455,2
2,1222,3
3,1090,3
4,1073,3
...,...,...
95,1593,5
96,710,2
97,2727,2
98,2279,2


Transformer Model

In [ ]:
# Model

#This function takes an input tensor x (image) and applies a convolutional layer to extract patch embeddings.
#The convolutional layer uses a kernel size specified by patch_size, and the resulting tensor is reshaped to flatten the spatial dimensions
def patch_embedding(x, patch_size=8, emb_size=128):
    x = layers.Reshape((-1, patch_size * patch_size * 3))(x)
    x = layers.Dense(emb_size)(x)
    return x




#This function implements the attention mechanism using dense layers for queries, keys, and values.
#The MultiHeadAttention layer processes the queries, keys, and values in parallel across multiple heads.
def attention(x, dim, n_heads, dropout=0.):
    q = layers.Dense(dim)(x) #Linear Transformed
    k = layers.Dense(dim)(x)
    v = layers.Dense(dim)(x)
    attn_output = layers.MultiHeadAttention(num_heads=n_heads, key_dim=dim, dropout=dropout)(q, k, v)
    return attn_output


#Implements a feed-forward layer with a ReLU activation function and dropout for regularization.
def feed_forward(x, dim, hidden_dim, dropout=0.): #transform attention weighted vectors
    x = layers.Dense(hidden_dim, activation='relu')(x)
    x = layers.Dropout(dropout)(x) #to avoid over fitting
    x = layers.Dense(dim)(x)
    x = layers.Dropout(dropout)(x)
    return x


#Adds a residual connection by adding the original input tensor x to the output of a given function fn(x).
def residual_add(x, fn): #add inputs before and after the function and add '(+)' them
    res = x
    x = fn(x)
    x += res
    return x


#Combines the attention mechanism and feed-forward layer using a residual connection and layer normalization.
def transformer_block(x, dim, n_heads, dropout=0.1):
    x = residual_add(x, lambda x: layers.LayerNormalization()(attention(x, dim, n_heads, dropout))) #Layer normalization not batch as we use sequences
    x = residual_add(x, lambda x: layers.LayerNormalization()(feed_forward(x, dim, dim, dropout)))
    return x

#The overall ViT model architecture, including patch embedding, positional encoding, transformer blocks, and output layer.
def vit_model(img_size=150, patch_size=3, emb_dim=32, n_layers=6, n_heads=2, out_dim=5, dropout=0.1):
    num_patches = (img_size // patch_size) ** 2

    inputs = layers.Input(shape=(img_size, img_size, 3))
    x = patch_embedding(inputs, patch_size, emb_dim)

    cls_token = tf.Variable(tf.random.normal((1, 1, emb_dim)))
    pos_embedding = tf.Variable(tf.random.normal((1, num_patches + 1, emb_dim)))
    cls_tokens = tf.tile(cls_token, [tf.shape(x)[0], 1, 1])
    x = tf.concat([cls_tokens, x], axis=1)
    x += pos_embedding[:, :(num_patches + 1)]

    for _ in range(n_layers):
        x = transformer_block(x, emb_dim, n_heads, dropout)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(out_dim, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model




# Instantiate the model
model = vit_model(img_size=150, patch_size=3, emb_dim=32, n_layers=6, n_heads=2, out_dim=5, dropout=0.1)

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss=losses.CategoricalCrossentropy(), metrics=['accuracy'])
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Train the model
model_f = model.fit(train,
                    validation_data=validation,
                    epochs=10,
                    batch_size=32,
                    callbacks=[early_stopping])
# Access training and validation loss
training_loss = model_f.history['loss']
validation_loss = model_f.history['val_loss']

# Plot training and validation loss
epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'bo', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Save the plot
plt.savefig('training_validation_loss_plot.png')

# Display the plot
plt.show()

# Save the trained model
model.save("transformer.h5")

# Display the model summary
model.summary()

In [ ]:
# Testing the model
tansformer_model = load_model('transformer.h5')
transformer_test_D_L  = test_model(tansformer_model , test)

transformer_Classified_Images = list(zip(images_names, transformer_test_D_L))
generate_csv(transformer_Classified_Images)


In [ ]:
formating('try')